### Q1. What is Gradient Boosting Regression?

Gradient Boosting Regression (GBR) is a powerful machine learning algorithm used for both regression and classification tasks. It is a type of boosting algorithm that involves iteratively adding weak learners to the model to improve its performance.

In Gradient Boosting Regression, the weak learners used are typically decision trees. The algorithm works by first fitting a decision tree to the data, then using the residuals (the difference between the predicted values and the true values) from that tree as the target variable for the next decision tree. This process is repeated until a specified number of trees have been added, or until the model's performance stops improving.

The "gradient" in Gradient Boosting Regression refers to the fact that the algorithm tries to minimize the loss function by iteratively descending the gradient of the function with respect to the model parameters. The gradient descent process involves adjusting the weights of the weak learners in each iteration to reduce the error in the model.

GBR is known for its ability to handle complex, non-linear relationships between the predictor variables and the response variable, as well as for its ability to handle missing data and outliers. However, it can be computationally expensive and prone to overfitting if not tuned properly.

### Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a
simple regression problem as an example and train the model on a small dataset. Evaluate the model's
performance using metrics such as mean squared error and R-squared.

In [6]:
import numpy as np

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.training_mean = None
        
    def fit(self, X, y):
        self.training_mean = np.mean(y)
        y_pred = np.full_like(y, self.training_mean)
        for i in range(self.n_estimators):
            residuals = y - y_pred
            tree = self.build_tree(X, residuals, self.max_depth)
            self.trees.append(tree)
            update = self.learning_rate * self.predict(X, tree)
            y_pred += update
            
    def build_tree(self, X, y, max_depth):
        if max_depth == 0:
            return np.mean(y)
        else:
            feature_idxs = np.random.choice(X.shape[1], size=1)
            best_feature_idx = feature_idxs[0]
            best_threshold = None
            best_loss = np.inf
            for i in feature_idxs:
                thresholds = np.unique(X[:, i])
                for threshold in thresholds:
                    left_idxs = X[:, i] < threshold
                    right_idxs = X[:, i] >= threshold
                    left_loss = self.compute_loss(y[left_idxs])
                    right_loss = self.compute_loss(y[right_idxs])
                    loss = left_loss + right_loss
                    if loss < best_loss:
                        best_feature_idx = i
                        best_threshold = threshold
                        best_loss = loss
            left_idxs = X[:, best_feature_idx] < best_threshold
            right_idxs = X[:, best_feature_idx] >= best_threshold
            left_subtree = self.build_tree(X[left_idxs, :], y[left_idxs], max_depth-1)
            right_subtree = self.build_tree(X[right_idxs, :], y[right_idxs], max_depth-1)
            return (best_feature_idx, best_threshold, left_subtree, right_subtree)
        
    def predict(self, X, tree=None):
        if tree is None:
            tree = self.trees[-1]
        if not isinstance(tree, tuple):
            return tree
        else:
            feature_idx, threshold, left_subtree, right_subtree = tree
            if X[feature_idx] < threshold:
                return self.predict(X, left_subtree)
            else:
                return self.predict(X, right_subtree)
            
    def compute_loss(self, y):
        return np.mean(np.square(y - np.mean(y)))
    
    def predict_all_trees(self, X):
        return np.sum([self.learning_rate * self.predict(X, tree) for tree in self.trees], axis=0) + self.training_mean


This code defines a class GradientBoostingRegressor that takes in several hyperparameters such as the number of estimators, learning rate, and maximum depth of each decision tree. The fit method trains the model on the input data X and target variable y using gradient boosting. The build_tree method constructs a decision tree by recursively splitting the data based on the feature that results in the lowest loss. The predict method predicts the output value for a given input using a decision tree. The compute_loss method calculates the mean squared error loss for a given set of target values. The predict_all_trees method computes the final prediction for an input by summing the predictions from all decision trees.

### Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to
optimise the performance of the model. Use grid search or random search to find the best
hyperparameters

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Split the data into training and testing sets
split_idx = int(0.7 * len(data))
X_train, y_train = data[:split_idx], target[:split_idx]
X_test, y_test = data[split_idx:], target[split_idx:]

# Train a gradient boosting regressor on the training data
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

# Predict the housing prices on the testing data
y_pred = gb.predict(X_test)

# Evaluate the model using mean squared error and R^2 score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error:", mse)
print("R^2 score:", r2)


Mean squared error: 52.67800904196831
R^2 score: 0.20601403739042323


In this code, we first load the Boston Housing dataset and split it into training and testing sets. We then define a grid of hyperparameters to search over, including the learning rate, number of estimators (trees), and maximum depth of each tree. We create a GridSearchCV object with the GradientBoostingRegressor as the estimator, and fit it to the training data to search for the best hyperparameters. We then create a new GradientBoostingRegressor object with the best hyperparameters found by the grid search, fit it to the training data, and make predictions on the testing data. Finally, we compute the mean squared error and R-squared score of the predictions and print them to the console.

Note that you can also use a random search instead of a grid search by replacing GridSearchCV with RandomizedSearchCV and adjusting the param_distributions parameter accordingly. Random search is often more efficient than grid search for high-dimensional hyperparameter spaces.

### Q4. What is a weak learner in Gradient Boosting?

In Gradient Boosting, a weak learner is a model that is only slightly better than random guessing. Weak learners are typically simple models, such as decision trees with a low maximum depth or linear regression models with few features. The idea behind using weak learners is that, by combining many of them, a strong predictor can be created. In each iteration of the boosting algorithm, a new weak learner is trained to correct the errors made by the previous learners. The output of each weak learner is then combined with the output of the previous learners to create a final prediction. The strength of the final model is determined by the number of weak learners used and their individual performance.

### Q5. What is the intuition behind the Gradient Boosting algorithm?

The intuition behind Gradient Boosting is to iteratively improve a weak model by adding a new model that corrects the errors of the previous model. In other words, it combines multiple weak models to create a strong model.

The algorithm works by first fitting a weak model to the data, typically a decision tree with only a few levels. It then calculates the difference between the actual target values and the predicted values from the weak model, and this difference becomes the target variable for the next weak model. The new weak model is then fitted to the residuals, and the process is repeated until the desired level of performance is achieved.

The key idea behind Gradient Boosting is that each weak model should be optimized to correct the errors of the previous model. This is achieved by using a gradient descent optimization algorithm to update the parameters of each weak model. The gradient descent algorithm minimizes a cost function, which is typically the mean squared error between the predicted and actual values. By iteratively optimizing the weak models, Gradient Boosting can create a strong model that is highly accurate at making predictions.

### Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?

The Gradient Boosting algorithm builds an ensemble of weak learners in a stage-wise manner. The process can be summarized as follows:

The first weak learner (usually a decision tree) is trained on the original data set.

The predictions of the first weak learner are used to identify the instances that the model has not predicted correctly. These instances are given a higher weight in the subsequent iteration.

In the subsequent iteration, a new weak learner is trained on the same data set with the adjusted weights.

The predictions of this new weak learner are combined with the predictions of the previous weak learner using a weighted sum. The weights are determined by the learning rate (a hyperparameter of the algorithm) and the performance of the new weak learner on the data set.

The process continues until a stopping criterion is met (e.g., a maximum number of iterations or the accuracy of the model reaches a certain threshold).

The final ensemble of weak learners is obtained by combining the predictions of all the weak learners using a weighted sum, where the weights are determined by the learning rate and the performance of the corresponding weak learner on the data set.

The intuition behind this process is that the subsequent weak learners are trained on the instances that the previous learners have not predicted correctly, so they can learn to correct the errors made by the previous learners. The final ensemble of weak learners is able to capture the complex non-linear relationships in the data set and make accurate predictions.

### Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting
algorithm?

The mathematical intuition behind Gradient Boosting algorithm can be broken down into the following steps:

Initialize the model with a constant value: The first step is to initialize the model with a constant value, usually the mean of the target variable. This constant value represents the initial prediction for all observations.

Compute the residuals: The next step is to compute the residuals by subtracting the predicted values from the actual target values. The residuals represent the difference between the true value and the predicted value.

Train a weak learner on the residuals: A weak learner is trained on the residuals to learn the patterns in the data that were not captured by the previous model. The weak learner is trained to minimize the residual error.

Update the model: The weak learner's prediction is added to the existing model, and this updated model is used to predict the target variable. The new predictions are computed by adding the weak learner's prediction to the previous predictions.

Repeat steps 2-4: The process is repeated until a stopping criterion is met, such as a maximum number of iterations or a minimum improvement in the loss function.

Combine the weak learners: Finally, the weak learners are combined to form the final prediction model. The final prediction is computed as the sum of the initial prediction and the predictions of all the weak learners.

The key idea behind Gradient Boosting algorithm is to iteratively improve the prediction by training a series of weak learners on the residuals of the previous model. By doing so, the algorithm is able to capture the complex interactions between the variables and improve the accuracy of the prediction.